In [ ]:
### imports, install
import sys
!{sys.executable} -m pip install  matplotlib pandas numpy pandasql openpyxl nx pydot graphviz

In [ ]:
### imports
import pandas as pd
import pandasql as pdsql

import math
import json

from ipyleaflet import Map, Marker, Polyline, AntPath


In [ ]:
### function to read excel into datagrame
def getDataFrame(pFileName, pColNameVector, pDictColValue):
    ds=open(pFileName, "rb")
    df=pd.read_excel(pFileName, index_col=False, names=pColNameVector, dtype = pDictColValue)
    ds.close()
    print(df)
    return df

In [ ]:
### read dataframe
## dataset 1 - routes
fileName= './data/ticketToRide.xlsx'
namesDs1=[ "city1", "distance",  "color", "tracks", "city2"]
dictType = {"city1":"str", "distance":"int", "color":"str", "tracks":"int", "city2":"str"}
df1=getDataFrame(fileName, namesDs1, dictType)

In [ ]:
#@dataset2: latitude & longitude
fileName= './data/ticketToRide-coords.xlsx'
namesDs1=[ "city1", "latitude",  "longitude"]
dictType = {"city1":"str", "latitude":"float", "longitude":"float"}
df2=getDataFrame(fileName, namesDs1, dictType)

In [ ]:
## merging dataframes
sqlJoin=""" select 
df1.city1 as orig
, df1.city2 as dest
, df1.distance
, df1.color
, loc1.latitude as orig_lat
, loc1.longitude as orig_lon
, loc2.latitude as dest_lat
, loc2.longitude as dest_lon

from 

df1  
inner join 
df2 loc1 
on df1.city1 = loc1.city1 

inner join
df2 loc2
on df1.city2=loc2.city1


"""
dfCitiesLocation = pdsql.sqldf(sqlJoin, locals())
print (dfCitiesLocation)

In [ ]:
#https://ipyleaflet.readthedocs.io/en/latest/api_reference/marker.html
class Location:
    def __init__(self, pLat=36.3, pLong=137.97):
        self.latitude =pLat
        self.longitude=pLong

In [ ]:
## New York
m = Map(center=(40, -74), zoom=6)

In [ ]:
## Display the map
m

In [ ]:
## print all routes
ant_paths = []
for route in dfCitiesLocation.itertuples():
    
    ## origin
    orig_name  = getattr(route, "orig")
    orig_lon=getattr(route, "orig_lon")
    orig_lat = getattr(route, "orig_lat")
    center_orig = (orig_lat, orig_lon )
    
    mark_orig = Marker(location=center_orig
                       , title =orig_name
                       , draggable=False
                      )
    m.add_layer(mark_orig)
    
    
    ## destination
    dest_name= getattr(route, "dest")
    dest_lon=getattr(route,"dest_lon")
    dest_lat = getattr(route, "dest_lat")
    center_dest = (dest_lat, dest_lon)
    
    mark_dest = Marker(location=center_dest
                       , title=dest_name
                       , draggable=False
                      )
    m.add_layer(mark_dest)
   
    
    
    routeColor = getattr(route, "color")
    distance = getattr(route,"distance")
    
    ### ant path
    #ant_paths.append(AntPath(locations=[[orig_lat, orig_lon ], [dest_lat, dest_lon]]
    #                         , use="polyline"
    #                        , weigth = 0.5 +distance
    #                        , color = routeColor))
    #m += ant_paths[-1]
    
    
    ### polyline
    line = Polyline(
        locations=[[orig_lat, orig_lon ], [dest_lat, dest_lon]]
        , weight = distance
        , color = routeColor
    )
    m.add_layer(line)
    
    
    print("orig: ", orig_name, " - dest: ", dest_name, " dist: ", distance, " color : ", routeColor)

In [120]:
%matplotlib notebook
%config IPCompleter.greedy=True
import networkx as nx
import matplotlib.pyplot as plt

%config IPCompleter.greedy=True
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image

In [121]:
multiGraph= nx.MultiGraph()

In [122]:
for edge in df1.itertuples():
    print(getattr(edge, "city1"), getattr(edge, "city2"))
    multiGraph.add_edge(getattr(edge, "city1"), getattr(edge, "city2")
                        , weight = getattr(edge, "distance")
                        , label = getattr(edge, "distance")
                        , penwidth = 1.2*getattr(edge, "distance")
                       , color = getattr(edge, "color"))



Vancouver Seattle
Vancouver Seattle
Vancouver Calgary
Seattle Portland
Seattle Calgary
Seattle Helena
Calgary Helena
Calgary Winnipeg
Portland San Francisco
Portland San Francisco
Portland Salt Lake City
Helena Salt Lake City
Helena Winnipeg
Helena Duluth
Helena Omaha
Helena Denver
Winnipeg Sault St Marie
Winnipeg Duluth
San Francisco Salt Lake City
San Francisco Salt Lake City
San Francisco Los Angeles
San Francisco Los Angeles
Salt Lake City Las Vegas
Salt Lake City Denver
Salt Lake City Denver
Duluth Sault St Marie
Duluth Toronto
Duluth Chicago
Duluth Omaha
Duluth Omaha
Omaha Denver
Omaha Kansas City
Omaha Kansas City
Omaha Chicago
Denver Kansas City
Denver Kansas City
Denver Phoenix
Denver Santa Fe
Denver Oklahoma City
Sault St Marie Montreal
Sault St Marie Toronto
Los Angeles Las Vegas
Los Angeles El Paso
Los Angeles Phoenix
Toronto Montreal
Toronto Pittsburgh
Toronto Chicago
Chicago Pittsburgh
Chicago Pittsburgh
Chicago St Louis
Chicago St Louis
Kansas City St Louis
Kansas City S

In [123]:
d = nx.drawing.nx_pydot.to_pydot(multiGraph) # d is a pydot graph object, dot options can be easily set
# attributes get converted from networkx,
# use set methods to control dot attributes after creation

png_str = d.create_png()
sio = BytesIO() # file-like string, appropriate for imread below
sio.write(png_str)
sio.seek(0)

img = mpimg.imread(sio)
plt.figure(figsize=(30,30))
imgplot = plt.imshow(img, aspect='auto')

FileNotFoundError: [Errno 2] "dot" not found in path.

In [124]:
df1.items

<bound method DataFrame.items of           city1  distance   color  tracks       city2
0     Vancouver         1    gray       2     Seattle
1     Vancouver         1    gray       2     Seattle
2     Vancouver         3    gray       1     Calgary
3       Seattle         1    gray       2    Portland
4       Seattle         4    gray       1     Calgary
..          ...       ...     ...     ...         ...
95  New Orleans         4  orange       2     Atlanta
96  New Orleans         6     red       1       Miami
97      Atlanta         2    gray       1  Charleston
98      Atlanta         5   black       1       Miami
99   Charleston         4    pink       1       Miami

[100 rows x 5 columns]>

In [125]:
sorted(nx.degree_centrality(multiGraph).items(), key=operator.itemgetter(1), reverse=True)[0:10]

NameError: name 'operator' is not defined